The velocity autocorrelation function (VACF) is given by:

$$C_v(t) = \frac{1}{N}\sum_i \sum_j \vec{v}_i(t)\cdot\vec{v}_j(0)$$

where $N$ is the number of atoms in the system, $\vec{v}_i(t)$ is the velocity of atom $i$ at time $t$, and the sum is taken over all atoms and all times $t$.

The static structure factor (SSF) is given by:

$$S(q) = \frac{1}{N}\sum_{j=1}^N \exp\left(-i\vec{q}\cdot\vec{r}_j\right)$$

where $N$ is the number of atoms in the system, $\vec{r}_j$ is the position of atom $j$, $\vec{q}$ is the wavevector, and the sum is taken over all atoms.

The dynamic structure factor (DSF) is given by:

$$S(q,\omega) = \frac{1}{N}\sum_{j=1}^N \sum_{k=1}^{Nt} \exp\left(-i\vec{q}\cdot\vec{r}_j\right) \exp\left(-i\omega t_k\right) \vec{v}_j(t_k)\cdot\vec{v}_k(0)$$

where $N$ is the number of atoms in the system, $\vec{r}_j$ is the position of atom $j$, $\vec{v}_j(t_k)$ is the velocity of atom $j$ at time $t_k$, $\vec{v}_k(0)$ is the velocity of atom $k$ at time $0$, $Nt$ is the number of simulation time steps, $\omega$ is the frequency, $\vec{q}$ is the wavevector, and the sums are taken over all atoms and all simulation time steps.

In [1]:
import numpy as np
from scipy.fftpack import fft, fftfreq
import matplotlib.pyplot as plt

In [ ]:
# Define the parameters of the simulation
n_atoms = 1000  # number of atoms in the system
box_size = 5.0  # size of the simulation box
dt = 0.001  # time step of the simulation
t_total = 1000  # total simulation time
n_steps = int(t_total / dt)  # number of simulation steps

# Define the variables needed for the calculations
q_max = np.pi / box_size
q_values = np.arange(0, q_max, 0.1)  # wavevector values for the static structure factor
omega_values = 2 * np.pi * fftfreq(n_steps, dt)  # frequency values for the dynamic structure factor

# Read in the position and velocity data from the LAMMPS dump file
data = np.genfromtxt('gold_block.dump', skip_header=9)

# Extract the positions and velocities of the atoms
positions = data[:, 2:5]
velocities = data[:, 5:]

# Calculate the velocity autocorrelation function (VACF)
vacf = np.zeros(n_steps)
for i in range(n_atoms):
    for j in range(n_steps):
        vacf[j] += np.dot(velocities[i + n_atoms*j], velocities[i])
vacf /= (n_atoms * n_steps)

# Calculate the static structure factor (SSF)
ssf = np.zeros(len(q_values))
for i, q in enumerate(q_values):
    for j in range(n_atoms):
        phase = np.dot(positions[j], q)
        ssf[i] += np.exp(-1j*phase)
ssf /= n_atoms

# Plot the static structure factor
plt.plot(q_values, np.abs(ssf)**2)
plt.xlabel('q')
plt.ylabel('S(q)')
plt.show()

# Calculate the dynamic structure factor (DSF)
dsf = np.zeros((len(q_values), len(omega_values)))
for i, q in enumerate(q_values):
    for j, omega in enumerate(omega_values):
        for k in range(n_atoms):
            phase = np.dot(positions[k], q)
            dsf[i, j] += np.exp(-1j*phase) * np.exp(-1j*omega*j*dt) * np.dot(velocities[k + n_atoms*j], velocities[k])
        dsf[i, j] /= (n_atoms * n_steps)

# Plot the dynamic structure factor
plt.imshow(np.log10(np.abs(dsf)), origin='lower', extent=[omega_values.min(), omega_values.max(), q_values.min(), q_values.max()], aspect='auto')
plt.colorbar()
plt.xlabel('omega')
plt.ylabel('q')
plt.show()
